In [77]:
greeting = '안녕'

In [78]:
greeting  # 바이트로 표시.

'\xec\x95\x88\xeb\x85\x95'

In [79]:
with open('./test.txt', 'w') as text:
    text.write(greeting)  # txt 확장자 파일은 유니코드 문자 입/출력이 가능하므로 '안녕'이 그대로 들어감.

파이썬3의 `str()`은 항상 유니코드 문자열을 리턴하지만 파이썬2는 경우가 다르다. <br>

In [80]:
def str(s):
    """Return a nice string representation of the object.  The
    return value is a str or unicode instance.
    """
    if type(s) is str or type(s) is unicode:
        return s
    r = s.__str__()
    if not isinstance(r, (str, unicode)):
        raise TypeError('__str__ returned non-string')
    return r

> In Python 2, objects can specify both a string and unicode representation of themselves. In Python 3, though, there is only a string representation. This becomes an issue as people can inadvertently do things in their __str__() methods which have unpredictable...
https://docs.python.org/3.3/howto/pyporting.html

## 인코딩 & 디코딩

인코딩은 codepoint를 byte로 변환하는 것을, 디코딩은 byte를 codepoint로 변환하는 것을 말한다.

코드포인트(code point)는 10진수 0부터 1,114,111 까지의 숫자를 범위로 갖는 문자의 단위원소다. <br>
유니코드 표준에서는 `U+` 접두사를 붙여서 4~6자리의 16진수료 표현한다. <br>
유니코드 6.3에서 가용한 코드포인트의 약 10퍼센트 정도가 문자에 할당되어 있다.
> 예 : 문자 A = U+0041

문자를 표현하는 실제 바이트는 사용하는 인코딩 코덱에 따라 달라진다. 

## UnicodeEncodeError & UnicodeDecodeError

### UnicodeEncodeError
대부분의 none-UTF 코덱은 유니코드 문자의 일부만 처리할 수 있다. <br>
텍스트를 바이트로 변환할 때 문자가 대상 코덱에 정의되어 있지 않으면 `UnicodeEncodeError`가 발생한다. <br>
`encode()`메서드의 `errors` 인수에 별도의 처리기를 지정한다면 발생하지 않을 수도 있다. 

In [81]:
city = u'서울'  
# 파이썬2에서 유니코드 문자열을 정의할 때는 문자열 앞에 'u'를 붙여서 unicode 객체를 만든다. u를 붙이지 않으면 디폴트로 ascii가 코덱으로 정해진다.
type(city)

unicode

In [82]:
city.encode('utf-8')

'\xec\x84\x9c\xec\x9a\xb8'

In [83]:
city.encode('utf-16')

'\xff\xfe\x1c\xc1\xb8\xc6'

In [84]:
city.encode('cp437')  # cp437 코덱은 한글을 인코딩할 수 없으므로 UnicodeEncodeError를 발생시킨다.

UnicodeEncodeError: 'charmap' codec can't encode characters in position 0-1: character maps to <undefined>

In [85]:
city.encode('cp437', errors='ignore') # 인코딩이 불가능한 문자는 건너뛰어버린다.

''

In [86]:
city.encode('cp437', errors='replace') # 인코딩이 불가능한 문자는 물음표로 치환한다.

'??'

### UnicodeDecodeError

codepoint를 byte로 인코딩한 후에, byte를 다시 codepoint로 디코딩할 경우, 모든 byte는 정당한 ascii문자가 될 수 없으며 모든 byte sequence가 정당한 UTF-8 또는
UTF-16 문자가 되는 것은 아니다. 따라서 binary sequence를 텍스트로 변환할 때 정당한 문자로 변환할 수 없으면 `UnicdeDecodeError`가 발생한다.

In [89]:
octet = b'Montr\xe9al'  # octet은 latin-1 코덱으로 인코딩된 문자열이다.
octet.decode('latin-1')

u'Montr\xe9al'

In [90]:
octet.decode('cp1252')  # cp1252는 latin-1 코덱의 상위집합이므로 정상적으로 디코딩된다.

u'Montr\xe9al'

In [92]:
octet.decode('utf-8')  # utf-8은 latin-1로 인코딩된 문자열을 디코딩할 수 없음을 알리기 위해 UnicodeDecodeError를 발생시킨다. (utf-8은 ascii와 호환된다.)

UnicodeDecodeError: 'utf8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

## 결론

* 유니코드 = 문자 정의 표준, UTF-8 = 문자를 인코딩, 디코딩하는 방식. <br> 예) 유니코드 문자 'A' 가 UTF-8 방식으로 인코딩 된다.
* 문자집합이 부족하거나 다른 코덱과 호환되지 않는 코덱은 인코딩과 디코딩을 거치는 과정에서 예외가 발생한다. <br>
* 또는 예외를 발생시키지 않기 위해 처리기를 집어넣을 경우 문자가 손실되는 경우가 있다. <br>
* 코드를 가장 많이 커버하는 `UTF-8`이 범용적으로 쓰인다. <br>
* python2 에서는 문자열의 기본 코덱이 `ascii`로 되어있으므로 `a = u'문자열'` 또는 `a = unicde('문자열')` 와 같이 문자열의 코덱을 유니코드로 지정해야 한다.